### Note
You can find the required data on the Nextcloud data/Paperdaten/2019/TGF2019-vadere

In [ ]:
# expand the cell of the notebook
import json
import gc
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import functools
import operator
sns.set_context("poster")
sns.set(style="whitegrid", font_scale=1.8)


from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

# Convert Vadere trajectories into a DataFrame

In [ ]:
def fs_append(pedestrianId, fs, llist):
    llist.append([pedestrianId, fs['start']['x'], fs['start']['y'], fs['startTime'], fs['end']['x'], fs['end']['y'], fs['endTime']])
   
def trajectory_append(pedestrianId, trajectory, llist):
    for fs in trajectory:
        fs_append(pedestrianId, fs, llist)

def trajectories_to_dataframe(trajectories):
    llist = []
    for pedId in trajectories:
        trajectory_append(pedId, trajectories[pedId], llist)
    dataframe = pd.DataFrame(llist, columns=['pedestrianId','startX','startY','startTime','endX','endY','endTime'])
    dataframe["distance"] = np.sqrt(np.square(dataframe["endX"] - dataframe["startX"]) + np.square(dataframe["endY"] - dataframe["startY"]))
    dataframe["velocity"] = dataframe["distance"] / (dataframe["endTime"] - dataframe["startTime"])
    return dataframe

In [ ]:
def load_simulation_data(file):
    f = open(file, "r")
    header = f.readline();
    trajectories = dict({});
    for row in f:
        s = row.split(" ");
        pedId = int(s[0]);
        footsteps = json.loads(s[1]);
        trajectories[pedId] = footsteps[0]['footSteps'];   
    return trajectories_to_dataframe(trajectories)

# Convert experiment data into a DataFrame

In [ ]:
def load_experiment(file):
    fps = 16
    data = pd.read_csv(
        file, 
        sep=' ', 
        names=['pedestrianId', 'timeStep', 'x', 'y', 'e'], 
        index_col=False, 
        header=None, 
        skiprows=0)
        
    rows = []
    #print(trajectories)
    last_ped_id = None
    lastX = None
    lastY = None
    for row in data.itertuples():
        endX = row.x / 100.0 + 18.7
        endY = row.y / 100.0 + 4.2
        startTime = row.timeStep / fps - 1/fps
        endTime = row.timeStep / fps
        if last_ped_id is None or last_ped_id != row.pedestrianId:
            startX = np.nan
            startY = np.nan
            distance = np.nan
            velocity = np.nan
        else:
            startX = lastX / 100.0 + 18.7
            startY = lastY / 100.0 + 4.2
            distance = np.sqrt(np.square(endX - startX) + np.square(endY - startY))
            velocity = distance / (endTime - startTime)
            rows.append([row.pedestrianId, startX, startY, endX, endY, startTime, endTime, distance, velocity])
        last_ped_id = row.pedestrianId
        lastX = row.x
        lastY = row.y
            
    dataframe = pd.DataFrame(rows, columns=['pedestrianId', 'startX', 'startY', 'endX', 'endY','startTime','endTime','distance','velocity'])
    return dataframe
    
def to_trajectories(data):
    trajectories = dict({})
    trajectory = []
    for i in range(len(data)-1):
        pedId = data['pedestrianId'][i]
        if pedId == data['pedestrianId'][i+1]:
            pedId = data['pedestrianId'][i]
            x1 = data['x'][i]
            y1 = data['y'][i]
            x2 = data['x'][i+1]
            y2 = data['y'][i+1]
            startTime = data['timeStep'][i] 
            endTime = data['timeStep'][i+1]
            fs = {'startTime':startTime, 'endTime': endTime, 'start':{'x':x1, 'y':y1}, 'end':{'x':x2, 'y':y2}}
            trajectory.append(fs)
        else:
            trajectories[pedId] = trajectory
            trajectory = []
            pedId = data['pedestrianId'][i]
    return trajectories

# Load all data

The following code loads the experiment data as well as the simulated data and transforms everything into the same format (data frame). The simulated trajectories are cut with respect to the camera bounds of the experiment.

In [ ]:
#times = np.linspace(4,10,10)
#euclid_d(get_trajectory(1), get_trajectory(1), times)
#to_trajectories(load_experiment(real_file))[1]

trajectories240o050o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-050-240/ko-240-050-240_combined_MB.txt")
trajectories240o060o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-060-240/ko-240-060-240_combined_MB.txt")
trajectories240o080o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-080-240/ko-240-080-240_combined_MB.txt")
trajectories240o100o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-100-240/ko-240-100-240_combined_MB.txt")
trajectories240o120o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-120-240/ko-240-120-240_combined_MB.txt")
trajectories240o150o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-150-240/ko-240-150-240_combined_MB.txt")
trajectories240o240o240 = load_experiment("./T-junction-experiment-trajectories-files/KO/ko-240-240-240/ko-240-240-240_combined_MB.txt")

trajectoriesReal = pd.concat([trajectories240o050o240, trajectories240o060o240, trajectories240o080o240, 
                              trajectories240o100o240, trajectories240o120o240, trajectories240o150o240, 
                              trajectories240o240o240], ignore_index=True)

# trajectories starting from left
cut_minX = trajectoriesReal[trajectoriesReal["endX"] < 15].groupby(["pedestrianId"])["startX"].min().max() + 0.12

# trajectories starting from right
cut_maxX = trajectoriesReal[trajectoriesReal["endX"] > 21].groupby(["pedestrianId"])["startX"].max().min() - 0.2

# trajectories ending at top
cut_maxY = trajectoriesReal.groupby(["pedestrianId"])["endY"].max().min() - 0.168

trajectories240o050o240 = cut(trajectories240o050o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o060o240 = cut(trajectories240o060o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o080o240 = cut(trajectories240o080o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o100o240 = cut(trajectories240o100o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o120o240 = cut(trajectories240o120o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o150o240 = cut(trajectories240o150o240, cut_minX, cut_maxX, cut_maxY)
trajectories240o240o240 = cut(trajectories240o240o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o050o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-050-240.txt")
osm_trajectories240o050o240 = cut(osm_trajectories240o050o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o060o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-060-240.txt")
osm_trajectories240o060o240 = cut(osm_trajectories240o060o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o080o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-080-240.txt")
osm_trajectories240o080o240 = cut(osm_trajectories240o080o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o100o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-100-240.txt")
osm_trajectories240o100o240 = cut(osm_trajectories240o100o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o120o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-120-240.txt")
osm_trajectories240o120o240 = cut(osm_trajectories240o120o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o150o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-150-240.txt")
osm_trajectories240o150o240 = cut(osm_trajectories240o150o240, cut_minX, cut_maxX, cut_maxY)

osm_trajectories240o240o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-OSM-240-240-240.txt")
osm_trajectories240o240o240 = cut(osm_trajectories240o240o240, cut_minX, cut_maxX, cut_maxY)

trajectoriesOSM = pd.concat([osm_trajectories240o050o240, osm_trajectories240o060o240, osm_trajectories240o080o240, osm_trajectories240o100o240, osm_trajectories240o120o240, osm_trajectories240o150o240, osm_trajectories240o240o240], ignore_index=True)

bhm_trajectories240o050o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-050-240.txt")
bhm_trajectories240o050o240 = cut(bhm_trajectories240o050o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o060o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-060-240.txt")
bhm_trajectories240o060o240 = cut(bhm_trajectories240o060o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o080o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-080-240.txt")
bhm_trajectories240o080o240 = cut(bhm_trajectories240o080o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o100o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-100-240.txt")
bhm_trajectories240o100o240 = cut(bhm_trajectories240o100o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o120o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-120-240.txt")
bhm_trajectories240o120o240 = cut(bhm_trajectories240o120o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o150o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-150-240.txt")
bhm_trajectories240o150o240 = cut(bhm_trajectories240o150o240, cut_minX, cut_maxX, cut_maxY)

bhm_trajectories240o240o240 = load_simulation_data("./T-junction-sim-trajectory-files/trajectories-BHM-240-240-240.txt")
bhm_trajectories240o240o240 = cut(bhm_trajectories240o240o240, cut_minX, cut_maxX, cut_maxY)

trajectoriesBHM = pd.concat([bhm_trajectories240o050o240, bhm_trajectories240o060o240, bhm_trajectories240o080o240, 
                             bhm_trajectories240o100o240, bhm_trajectories240o120o240, bhm_trajectories240o150o240, 
                             bhm_trajectories240o240o240], ignore_index=True)

In [ ]:
# Sum up all measured time deltas of a pedestrian to get the final evacuation time
copy = trajectoriesReal.copy(deep=True)
copy["timeDelta"] = copy["endTime"] - copy["startTime"]
et = copy.groupby(["pedestrianId"])["timeDelta"].sum()
print("Evacuation time (real data)")
print("- mean: {:.2f} [s]".format(et.mean()))
print("- std: {:.2f} [s]".format(et.std()))
print("- min: {:.2f} [s]".format(et.min()))
print("- max: {:.2f} [s]".format(et.max()))
print("- minX: {:.2f} [m]".format(cut_minX))
print("- maxX: {:.2f} [m]".format(cut_maxX))
print("- maxY: {:.2f} [m]".format(cut_maxY))

# Convert DataFrame to postvis DataFrame

In [ ]:
def to_postVis(df):
    simTimeStep = 0.4
    fps = 16
    df['timeStep'] = np.ceil(df['endTime'] / (1/fps)).astype(np.int)
    df['x'] = df['endX']
    df['y'] = df['endY']
    df['simTime'] = df['endTime']
    df = df.drop(columns=['startX','startY','endX','endY','startTime', 'endTime'])    
    return df

## Simulation data

TODO: Use `PedestrianEvacuationTimeProcessor` to log evacuation time during simulation and analyze it here.

# Helper method to access parts of the trajectory

In [ ]:
def get_trajectory(pedId, trajectories):
    """returns a data frame containing the trajectory of one specific agent."""
    query = 'pedestrianId == ' + str(pedId)
    return trajectories.query(query)

def get_trajectories(t, trajectories):
    return trajectories[np.logical_and(trajectories.startTime <= t, trajectories.endTime > t)]

def get_pedestrianIds(trajectories):
    return trajectories['pedestrianId'].unique()

#def get_velocity(trajectories, t, dt):
#    trajectories[np.logical_and(trajectory.endX >= xmax, trajectory.startX < xmax)]

def get_footstep(trajectory, i):
    """returns the i-ths footstep."""
    return trajectory.iloc[i];

def get_footstep_by_time(trajectory, time):
    """returns the footstep which happens at time or nothing (None)."""
    query = 'startTime <= ' + str(time) + ' and ' + str(time) + ' < endTime'
    fs = trajectories.query(query)
    assert len(fs) >= 1
    return fs

def start_time(trajectory):
    """returns the time of the first footstep of the trajectory."""
    return get_footstep(trajectory, 0)['startTime'];

def end_time(trajectory):
    return get_footstep(trajectory, len(trajectory)-1)['endTime'];

def max_start_time(trajectories):
    """returns the time of the first footstep of the trajectory which starts last."""
    pedestrianIds = get_pedestrianIds(trajectories)
    return max(map(lambda pedId: start_time(get_trajectory(pedId, trajectories)), pedestrianIds))

def min_end_time(trajectories):
    """returns the time of the last footstep of the trajectory which ends first."""
    pedestrianIds = get_pedestrianIds(trajectories)
    return min(map(lambda pedId: end_time(get_trajectory(pedId, trajectories)), pedestrianIds))

def footstep_is_between(fs, time):
    """true if the foostep and the intersection with time is not empty."""
    startTime = fs['startTime'];
    endTime = fs['endTime'];
    return startTime <= time and time < endTime;

def cut(trajectory, sTime, eTime):
    query = 'startTime >= ' + str(sTime) + ' and endTime < ' + str(eTime)
    return trajectory.query(query)

def cut_soft(trajectory, sTime, eTime):
    query = 'endTime > ' + str(sTime) + ' and startTime < ' + str(eTime)
    return trajectory.query(query)

def cuthead_trajectory_by(trajectory, ymin, ymax):
    i1 = trajectory[trajectory.endY >= ymax].index.min()
    i2 = trajectory[trajectory.endY <= ymin].index.min()
    #assert (i1 is np.nan and i2 is not np.nan) or (i1 is not np.nan and i2 is np.nan)
    y = ymax if i2 is np.nan or (i1 is not np.nan and i1 < i2) else ymin
    i = i1 if y == ymax else i2
    if i is np.nan:
        print("i1:"+str(i1))
        print("i2:"+str(i2))
        
    assert i is not np.nan
    #print(i)
    # cut the footstep at the tail to exactly fit xmin or xmax
    fs = trajectory.loc[i]
    start = np.array([fs["startX"], fs["startY"]])
    end = np.array([fs["endX"], fs["endY"]])
    endTime = fs["endTime"]
    startTime = fs["startTime"]
    distance = fs["distance"]
    velocity = fs["velocity"]
    d = end - start
    if abs(fs["endY"] - fs["startY"]) > 0.00001:
        r = (y - fs["startY"]) / (fs["endY"] - fs["startY"])
        end = start + (d * r)
        time = fs["endTime"] - fs["startTime"]
        endTime = fs["startTime"] + (time * r)
        distance = np.linalg.norm(end - start)
        velocity = distance / (endTime - startTime)
        
    df = trajectory.loc[:i-1]
    llist = [[fs["pedestrianId"],fs["startX"],fs["startY"],fs["startTime"],end[0],end[1],endTime,distance,velocity]]
    df_head = pd.DataFrame(llist, columns=['pedestrianId','startX','startY','startTime','endX','endY','endTime','distance','velocity'])
    df = df.append(df_head, ignore_index=True)
    return df

def cuttail_trajectory_by(trajectory, xmin, xmax):
    #i1 = trajectory[np.logical_and(trajectory.endX >= xmax, trajectory.startX < xmax)].index.max()
    i1 = trajectory[trajectory.startX >= xmax].index.max()
    i2 = trajectory[trajectory.startX <= xmin].index.max()
    #assert (i1 is np.nan and i2 is not np.nan) or (i1 is not np.nan and i2 is np.nan)
    x = xmax if i2 is np.nan or (i1 is not np.nan and i1 > i2) else xmin
    i = i1 if x == xmax else i2
    assert i is not np.nan
    # cut the footstep at the tail to exactly fit xmin or xmax
    fs = trajectory.loc[i]
    start = np.array([fs["startX"], fs["startY"]])
    end = np.array([fs["endX"], fs["endY"]])
    startTime = fs["startTime"]
    endTime = fs["endTime"]
    distance = fs["distance"]
    velocity = fs["velocity"]
    d = end - start
    if abs(fs["endX"] - fs["startX"]) > 0.00001:
        r = (fs["endX"] - x) / (fs["endX"] - fs["startX"])
        start = end - (d * r)
        time = fs["endTime"] - fs["startTime"]
        startTime = fs["endTime"] - (time * r)
        distance = np.linalg.norm(end - start)
        velocity = distance / (endTime - startTime)
        assert startTime <= endTime
        
    df = trajectory.loc[i+1:]
    llist = [[fs["pedestrianId"],start[0],start[1],startTime,fs["endX"],fs["endY"],fs["endTime"],distance,velocity]]
    df_tail = pd.DataFrame(llist, columns=['pedestrianId','startX','startY','startTime','endX','endY','endTime','distance','velocity'])
    df_tail = df_tail.append(df, ignore_index=True)
    return df_tail

def cuthead_by(trajectories, ymin, ymax):
    df = pd.DataFrame([], columns=['pedestrianId','startX','startY','startTime','endX','endY','endTime'])
    pedIds = get_pedestrianIds(trajectories)
    for pedId in pedIds:
        df = df.append(cuthead_trajectory_by(get_trajectory(pedId, trajectories), ymin, ymax), ignore_index=True)
    return df

def cuttail_by(trajectories, xmin, xmax):
    df = pd.DataFrame([], columns=['pedestrianId','startX','startY','startTime','endX','endY','endTime'])
    pedIds = get_pedestrianIds(trajectories)
    for pedId in pedIds:
        df = df.append(cuttail_trajectory_by(get_trajectory(pedId, trajectories), xmin, xmax), ignore_index=True)
    return df

def cut(trajectories, cut_minX, cut_maxX, cut_maxY):
    df = cuttail_by(trajectories, cut_minX, cut_maxX)
    df = cuthead_by(df, -1000, cut_maxY)
    return df

# Helper methods to compute different metrices

In [ ]:
def footstep_length(fs):
    """Euclidean length of a footstep."""
    x1 = fs['startX'];
    y1 = fs['startY'];
    x2 = fs['endX'];
    y2 = fs['endY'];
    dx = x1-x2;
    dy = y1-y2;
    return np.sqrt(dx*dx + dy*dy);

def mean_velocity_at(t, trajectories):
    return get_trajectories(t, trajectories)['velocity'].mean()

def evacuation_times(trajectories):
    pedIds = get_pedestrianIds(trajectories)
    rows = []
    for pedId in pedIds:
        evacTime = evacuation_time(pedId, trajectories)
        rows.append([pedId, evacTime])
    return pd.DataFrame(rows, columns=['pedestrianId', 'evacuationTime'])
    
def evacuation_time(pedId, trajectories):
    traj = get_trajectory(pedId, trajectories)
    start = traj.iloc[0]['endTime']
    end = traj.iloc[len(traj)-1]['endTime']
    return end - start

def trajectory_length(trajectory):
    """Euclidean length of a trajectory."""
    dx = trajectory['startX']-trajectory['endX']
    dy = trajectory['startY']-trajectory['endY']
    return np.sqrt(dx*dx + dy*dy).sum();

def footstep_direction(fs):
    """Vector from start to end position."""
    x1 = fs['startX'];
    y1 = fs['startY'];
    x2 = fs['endX'];
    y2 = fs['endY'];
    return np.array([x2-x1, y2-y1]);

def footstep_duration(fs):
    """Duration of a footstep."""
    startTime = fs['startTime'];
    endTime = fs['endTime'];
    return endTime-startTime;

def trajectory_duration(trajectory):
    """Euclidean length of a trajectory."""
    return (trajectory['endTime'] - trajectory['startTime']).sum();

def footstep_speed(fs):
    """Speed of the footstep."""
    return footstep_length(fs) / footstep_duration(fs);

def trajectory_speed(fs):
    """Speed of the trajectory."""
    return trajectory_length(fs) / trajectory_duration(fs);

#def trajectory_positions(trajectory, times):
#    mask = trajectory[['startTime', 'endTime']].mask(lambda x: x**2)
#    (df['date'] > '2000-6-1') & (df['date'] <= '2000-6-10')
#    duration = trajectory['endTime'] - trajectory['startTime']
#    dx = trajectory['endX'] - trajectory['startX']
#    dy = trajectory['endY'] - trajectory['startY']
#    direction = 
    
def filter_trajectories(trajectories, times):
    """Filters trajectory by times."""
    rows = []
    for row in trajectories.itertuples():
        if len(list(filter(lambda b: b, map(lambda t: row.startTime <= t and t < row.endTime, times)))) > 0:
            rows.append(row)
    return pd.DataFrame(rows)

def trajectories_position(trajectories, times):
    """Transforms trajectories into positions at each time in times such that each position is computed by linear interpolation."""
    rows = []
    #print(trajectories)
    for row in trajectories.itertuples():
        llist = list(filter(lambda t: row.startTime <= t and t < row.endTime, times))
        assert len(llist) == 1 or len(llist) == 0
        if len(llist) > 0:
            time = llist[0]
            dur = row.endTime - row.startTime
            partial_dur = time - row.startTime
            ratio = partial_dur / dur
            direction = np.array([row.endX - row.startX, row.endY - row.startY])
            l = np.linalg.norm(direction)
            if l > 0:
                partial_l = l * ratio;
                v = direction / l * partial_l;
                pos = np.array([row.startX, row.startY]) + v;
                rows.append([row.pedestrianId, pos[0], pos[1], time])
            else:
                rows.append([row.pedestrianId, np.nan, np.nan, time])
    dataframe = pd.DataFrame(rows, columns=['pedestrianId','x','y','time'])
    return dataframe
    
def euclid_d(trajPos1, trajPos2):
    """Computes the total (Euclidean) distance between two trajectories.
       Assumption: trajectories are both cut acccordingly!   
    """
    assert len(trajPos1) == len(trajPos2)
    dx = trajPos1['x'] - trajPos2['x']
    dy = trajPos1['y'] - trajPos2['y']
    norm = np.sqrt(dx**2 + dy**2)
    return norm.sum() / len(dx)

def euclid_path_length(trajPos1, trajPos2):
    """Computes the total (Euclidean) path length difference between two trajectories.
       Assumption: trajectories are both cut acccordingly!
    """
    count = len(trajPos1)
    pad = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], columns=['pedestrianId','x','y','time'])
    trajPos1Pad = pd.concat([pad, trajPos1], ignore_index=True)
    trajPos2Pad = pd.concat([pad, trajPos1], ignore_index=True)
    dx1 = trajPos1['x'] - trajPos1Pad['x']
    dy1 = trajPos1['y'] - trajPos1Pad['y']
    dx2 = trajPos2['x'] - trajPos2Pad['x']
    dy2 = trajPos2['y'] - trajPos2Pad['y']
    dx = dx1 - dx2
    dy = dy1 - dy2
    diff = np.sqrt(dx**2 + dy**2)
    return diff.sum()

def euclid_len(trajectory, sTime, eTime):
    """Computes the total (Euclidean) length of the trajectory in between [sTime;eTime]."""
    cut_traj = cut_soft(trajectory, sTime, eTime);
    return trajectory_length(cut_traj)
    
def inter_agent_d(trajPos):
    """Computes the inter agent (Euclidean) distance between all pairs of agents.
       Assumption: the trajectory is cut accordingly, ie the time is equal for
       each position.
    """
    s = 0
    min_index = min(trajectories.keys())
    c = 0
    llen = len(trajPos)
    for index1, row1 in trajPos.iterrows():
        for index2, row2 in trajPos.tail(llen-1-index1).iterrows():
            x1 = row1['x']
            y1 = row1['y']
            x2 = row2['x']
            y2 = row2['y']
            dx = x1 - x2
            dy = y1 - y2
            s = s + np.sqrt(dx**2 + dy**2)
            c = c + 1
    if c == 0:
        return 0
    else:
        return s / c
    
def total_inter_agent(trajectories1, trajectories2, times):
    """too expensive! TODO!"""
    return sum(map(lambda t: inter_agent_d(trajectories_position(trajectories1, [t])) - inter_agent_d(trajectories_position(trajectories2, [t])), times)) / len(times)


In [ ]:
trajPos1 = trajectories_position(get_trajectory(2, osm_trajectories240o120o240), [1,2,3,4,5,6,8,9,10,11,12,13])
trajPos2 = trajectories_position(get_trajectory(7, osm_trajectories240o120o240), [1,2,3,4,5,6,8,9,10,11,12,13])
trajPos1 = trajPos1[~np.isnan(trajPos1.x)]
trajPos2 = trajPos2[~np.isnan(trajPos2.x)]
euclid_path_length(trajPos1, trajPos2)
euclid_len(osm_trajectories240o120o240,0,10000)
t = 0.5
ttraj = osm_trajectories240o120o240[np.logical_and(osm_trajectories240o120o240.startTime <= t, osm_trajectories240o120o240.endTime >= t)]
get_trajectories(0.5, osm_trajectories240o120o240).head()
#osm_trajectories240o120o240

In [ ]:
def greedy_match(trajectories1, trajectories2, times, f):
    """Computes a match of trajectories by using a greedy algorithm."""
    assert len(trajectories1) == len(trajectories2)
    min_index1 = min(trajectories1.keys())
    min_index2 = min(trajectories2.keys())
    match = {}
    indexSet = set(range(min_index2, len(trajectories2)))
    for i in range(min_index1, len(trajectories1)):
        traj1 = trajectories1[i]
        minVal = None
        minIndex = None
        for j in indexSet:
            traj2 = trajectories2[j]
            if overlap(traj1, traj2, 0.4):
                val = f(traj1, traj2, times)
                if(minVal == None or val < minVal):
                    minIndex = j
                    minVal = val
        match[i] = minIndex
        indexSet.remove(minIndex)
    return match

# Plot trajectories

Now we plot the cut trajectories.

In [ ]:
def to_line(trajectory, xleft):
    """Transforms a trajectory into a Line2D."""
    current_palette = sns.color_palette()
    x = trajectory['endX'].values
    y = trajectory['endY'].values
    if x[0] < xleft:
        c = current_palette[2]
    else:
        c = current_palette[0]
    return x, y, Line2D(x, y, color=c, linewidth=0.4)

def add_lines(trajectories, xleft, ax):
    grouped = trajectories.groupby(['pedestrianId'])
    for name, group in grouped:
        x, y, line = to_line(group, xleft)
        ax.add_line(line)
        
def contains(x,y,rect):
    #ma = mpl.patches.Rectangle((16.3,6.0), 2.4, 2.0)
    return x >= rect.get_x() and y >= rect.get_y() and x <= rect.get_x() + rect.get_width() and y <= rect.get_y() + rect.get_height()

def filter_by_time_and_place(t, rect, trajectories):
    """returns a subset of trajectories i.e. at most one footstep for each pedestrian / agent such that the footstep the position (x,y) is the position of the
    agent at the time t contained in the rectanlge rect. Two new colums will be added for x and y."""
    traj = get_trajectories(t, trajectories)
    #TODO: this is very very memory expensive!
    traj.loc[:,'x'] = traj.loc[:,'startX'] + (traj.loc[:,'endX'] - traj.loc[:,'startX']) * (t - traj.loc[:,'startTime']) / (traj.loc[:,'endTime'] - traj.loc[:,'startTime'])
    traj.loc[:,'y'] = traj.loc[:,'startY'] + (traj.loc[:,'endY'] - traj.loc[:,'startY']) * (t - traj.loc[:,'startTime']) / (traj.loc[:,'endTime'] - traj.loc[:,'startTime'])
    #traj.loc[:,'x'] = traj.loc[:,'startX']
    #traj.loc[:,'y'] = traj.loc[:,'startY']
    traj = traj[traj.apply(lambda x: contains(x['x'], x['y'],rect), axis=1)]
    return traj

def density_velocity(t, rect, trajectories):
    area = rect.get_width() * rect.get_height()
    traj = filter_by_time_and_place(t, rect, trajectories)
    meanVelocity = traj.loc[:,'velocity'].mean();
    number_of_peds = len(traj)
    traj = None
    #gc.collect()
    if number_of_peds == 0:
        return np.nan, np.nan
    else:
        return number_of_peds / area, meanVelocity

def density(t, rect, trajectories):
    area = rect.get_width() * rect.get_height()
    traj = filter_by_time_and_place(t, rect, trajectories)
    number_of_peds = len(traj)
    traj = None
    #gc.collect()
    if number_of_peds == 0:
        return np.nan
    else:
        return number_of_peds / area

In [ ]:
x_vcenter = 17.5
y_vcenter = 5.2

xmin = x_vcenter-4
xmax = x_vcenter+5
ymin = y_vcenter-3.2
ymax = y_vcenter+3

alp = 0.549020
measurementArea_front = mpl.patches.Rectangle((16.3,6.0), 2.4, 2.0, color='r', alpha = alp)
measurementArea_left = mpl.patches.Rectangle((14.2,1.8), 2.0, 2.4, color='r', alpha = alp)
measurementArea_right = mpl.patches.Rectangle((19.7,1.8), 2.0, 2.4, color='r', alpha = alp)

fig_trajectories = plt.figure(figsize=(20,20))
ax1_trajectories = fig_trajectories.add_subplot(131)
ax1_trajectories.add_patch(measurementArea_left)
ax1_trajectories.add_patch(measurementArea_right)
ax1_trajectories.add_patch(measurementArea_front)
add_lines(trajectories240o150o240, 16, ax1_trajectories)
ax1_trajectories.set_title("Experiment")
ax1_trajectories.set_xlim(xmin, xmax)
ax1_trajectories.set_ylim(ymin, ymax)
ax1_trajectories.set_aspect(1)

ax2_trajectories = fig_trajectories.add_subplot(132, sharey=ax1_trajectories)
ax2_trajectories.add_patch(mpl.patches.Rectangle((16.3,6.0), 2.4, 2.0, color='r', alpha = alp))
ax2_trajectories.add_patch(mpl.patches.Rectangle((14.2,1.8), 2.0, 2.4, color='r', alpha = alp))
ax2_trajectories.add_patch(mpl.patches.Rectangle((19.7,1.8), 2.0, 2.4, color='r', alpha = alp))
add_lines(osm_trajectories240o150o240, 16, ax2_trajectories)
plt.setp(ax2_trajectories.get_yticklabels(), visible=False)
ax2_trajectories.set_title("OSM")
ax2_trajectories.set_xlim(xmin, xmax)
ax2_trajectories.set_ylim(ymin, ymax)
ax2_trajectories.set_aspect(1)

ax3_trajectories = fig_trajectories.add_subplot(133, sharey=ax2_trajectories)
ax3_trajectories.add_patch(mpl.patches.Rectangle((16.3,6.0), 2.4, 2.0, color='r', alpha = alp))
ax3_trajectories.add_patch(mpl.patches.Rectangle((14.2,1.8), 2.0, 2.4, color='r', alpha = alp))
ax3_trajectories.add_patch(mpl.patches.Rectangle((19.7,1.8), 2.0, 2.4, color='r', alpha = alp))
add_lines(bhm_trajectories240o150o240, 16, ax3_trajectories)
plt.setp(ax3_trajectories.get_yticklabels(), visible=False)
ax3_trajectories.set_title("BHM")
ax3_trajectories.set_xlim(xmin, xmax)
ax3_trajectories.set_ylim(ymin, ymax)
ax3_trajectories.set_aspect(1)
plt.savefig('./trajectories.pdf', bbox_inches='tight')

# Computation of evacuation times

In [ ]:
#240-050-240
evacTime240o050o240 = evacuation_times(trajectories240o050o240)
evacTime240o050o240['scenario'] = '$b_{in} = 50$'
evacTime240o050o240['model'] = 'Experiment'

osm_evacTime240o050o240 = evacuation_times(osm_trajectories240o050o240)
osm_evacTime240o050o240['scenario'] = '$b_{in} = 50$'
osm_evacTime240o050o240['model'] = 'OSM'

bhm_evacTime240o050o240 = evacuation_times(bhm_trajectories240o050o240)
bhm_evacTime240o050o240['scenario'] = '$b_{in} = 50$'
bhm_evacTime240o050o240['model'] = 'BHM'

#240-060-240
evacTime240o060o240 = evacuation_times(trajectories240o060o240)
evacTime240o060o240['scenario'] = '$b_{in} = 60$'
evacTime240o060o240['model'] = 'Experiment'

osm_evacTime240o060o240 = evacuation_times(osm_trajectories240o060o240)
osm_evacTime240o060o240['scenario'] = '$b_{in} = 60$'
osm_evacTime240o060o240['model'] = 'OSM'

bhm_evacTime240o060o240 = evacuation_times(bhm_trajectories240o060o240)
bhm_evacTime240o060o240['scenario'] = '$b_{in} = 60$'
bhm_evacTime240o060o240['model'] = 'BHM'

#240-080-240
evacTime240o080o240 = evacuation_times(trajectories240o080o240)
evacTime240o080o240['scenario'] = '$b_{in} = 80$'
evacTime240o080o240['model'] = 'Experiment'

osm_evacTime240o080o240 = evacuation_times(osm_trajectories240o080o240)
osm_evacTime240o080o240['scenario'] = '$b_{in} = 80$'
osm_evacTime240o080o240['model'] = 'OSM'

bhm_evacTime240o080o240 = evacuation_times(bhm_trajectories240o080o240)
bhm_evacTime240o080o240['scenario'] = '$b_{in} = 80$'
bhm_evacTime240o080o240['model'] = 'BHM'

#240-100-240
evacTime240o100o240 = evacuation_times(trajectories240o100o240)
evacTime240o100o240['scenario'] = '$b_{in} = 100$'
evacTime240o100o240['model'] = 'Experiment'

osm_evacTime240o100o240 = evacuation_times(osm_trajectories240o100o240)
osm_evacTime240o100o240['scenario'] = '$b_{in} = 100$'
osm_evacTime240o100o240['model'] = 'OSM'

bhm_evacTime240o100o240 = evacuation_times(bhm_trajectories240o100o240)
bhm_evacTime240o100o240['scenario'] = '$b_{in} = 100$'
bhm_evacTime240o100o240['model'] = 'BHM'

#240-120-240
evacTime240o120o240 = evacuation_times(trajectories240o120o240)
evacTime240o120o240['scenario'] = '$b_{in} = 120$'
evacTime240o120o240['model'] = 'Experiment'

osm_evacTime240o120o240 = evacuation_times(osm_trajectories240o120o240)
osm_evacTime240o120o240['scenario'] = '$b_{in} = 120$'
osm_evacTime240o120o240['model'] = 'OSM'

bhm_evacTime240o120o240 = evacuation_times(bhm_trajectories240o120o240)
bhm_evacTime240o120o240['scenario'] = '$b_{in} = 120$'
bhm_evacTime240o120o240['model'] = 'BHM'

#240-150-240
evacTime240o150o240 = evacuation_times(trajectories240o150o240)
evacTime240o150o240['scenario'] = '$b_{in} = 150$'
evacTime240o150o240['model'] = 'Experiment'

osm_evacTime240o150o240 = evacuation_times(osm_trajectories240o150o240)
osm_evacTime240o150o240['scenario'] = '$b_{in} = 150$'
osm_evacTime240o150o240['model'] = 'OSM'

bhm_evacTime240o150o240 = evacuation_times(bhm_trajectories240o150o240)
bhm_evacTime240o150o240['scenario'] = '$b_{in} = 150$'
bhm_evacTime240o150o240['model'] = 'BHM'

#240-240-240
evacTime240o240o240 = evacuation_times(trajectories240o240o240)
evacTime240o240o240['scenario'] = '$b_{in} = 240$'
evacTime240o240o240['model'] = 'Experiment'

osm_evacTime240o240o240 = evacuation_times(osm_trajectories240o240o240)
osm_evacTime240o240o240['scenario'] = '$b_{in} = 240$'
osm_evacTime240o240o240['model'] = 'OSM'

bhm_evacTime240o240o240 = evacuation_times(bhm_trajectories240o240o240)
bhm_evacTime240o240o240['scenario'] = '$b_{in} = 240$'
bhm_evacTime240o240o240['model'] = 'BHM'

# merge all together
experiments_evacuation_times = pd.concat([evacTime240o240o240, evacTime240o150o240, evacTime240o120o240, evacTime240o100o240, evacTime240o080o240, evacTime240o060o240, evacTime240o050o240], ignore_index=True)
osm_evacuation_times = pd.concat([osm_evacTime240o240o240, osm_evacTime240o150o240, osm_evacTime240o120o240, osm_evacTime240o100o240, osm_evacTime240o080o240, osm_evacTime240o060o240, osm_evacTime240o050o240], ignore_index=True)
bhm_evacuation_times = pd.concat([bhm_evacTime240o240o240, bhm_evacTime240o150o240, bhm_evacTime240o120o240, bhm_evacTime240o100o240, bhm_evacTime240o080o240, bhm_evacTime240o060o240, bhm_evacTime240o050o240], ignore_index=True)
all_evacuation_times = pd.concat([experiments_evacuation_times, osm_evacuation_times, bhm_evacuation_times], ignore_index=True)

# Compute densities, velocities and evacuation times

In [ ]:
def compute_overall_velocity(times, trajectories):
    velocity_all = list(map(lambda t: mean_velocity_at(t, trajectories), times))
    veloicty_std_all = list(map(lambda t: get_trajectories(t, trajectories)['velocity'].std(),times))
    return velocity_all, veloicty_std_all

def compute_fundamentals(times, measurementAreas, trajectories):
    fundamentals_all = []
    for measurementArea in measurementAreas:
        densities_velocities = *map(lambda t: density_velocity(t, measurementArea, trajectories), times),
        density = list(map(lambda e: e[0], densities_velocities))
        velocity = list(map(lambda e: e[1], densities_velocities))
        fundamentals_all.append([density, velocity])
    return fundamentals_all

## Front
Computation of the density and velocity at the front measurement area.

In [ ]:
gc.disable()
times = np.arange(0, 110, 2)
measurementArea_front = mpl.patches.Rectangle((16.3,6.0), 2.4, 2.0)
measurementArea_left = mpl.patches.Rectangle((14.2,1.8), 2.0, 2.4)
measurementArea_right = mpl.patches.Rectangle((19.7,1.8), 2.0, 2.4)
measurementAreas = [measurementArea_front, measurementArea_left, measurementArea_right]

#240-050-240
fundamentals240o050o240_all = compute_fundamentals(times, measurementAreas, trajectories240o050o240)
velocities240o050o240 = compute_overall_velocity(times, trajectories240o050o240)

osm_fundamentals240o050o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o050o240)
osm_velocities240o050o240 = compute_overall_velocity(times, osm_trajectories240o050o240)

bhm_fundamentals240o050o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o050o240)
bhm_velocities240o050o240 = compute_overall_velocity(times, bhm_trajectories240o050o240)

gc.collect()

#240-060-240
fundamentals240o060o240_all = compute_fundamentals(times, measurementAreas, trajectories240o060o240)
velocities240o060o240 = compute_overall_velocity(times, trajectories240o060o240)

osm_fundamentals240o060o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o060o240)
osm_velocities240o060o240 = compute_overall_velocity(times, osm_trajectories240o060o240)

bhm_fundamentals240o060o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o060o240)
bhm_velocities240o060o240 = compute_overall_velocity(times, bhm_trajectories240o060o240)

gc.collect()

#240-080-240
fundamentals240o080o240_all = compute_fundamentals(times, measurementAreas, trajectories240o080o240)
velocities240o080o240 = compute_overall_velocity(times, trajectories240o080o240)

osm_fundamentals240o080o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o080o240)
osm_velocities240o080o240 = compute_overall_velocity(times, osm_trajectories240o080o240)

bhm_fundamentals240o080o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o080o240)
bhm_velocities240o080o240 = compute_overall_velocity(times, bhm_trajectories240o080o240)

gc.collect()

#240-100-240
fundamentals240o100o240_all = compute_fundamentals(times, measurementAreas, trajectories240o100o240)
velocities240o100o240 = compute_overall_velocity(times, trajectories240o100o240)

osm_fundamentals240o100o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o100o240)
osm_velocities240o100o240 = compute_overall_velocity(times, osm_trajectories240o100o240)

bhm_fundamentals240o100o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o100o240)
bhm_velocities240o100o240 = compute_overall_velocity(times, bhm_trajectories240o100o240)

gc.collect()

#240-120-240
fundamentals240o120o240_all = compute_fundamentals(times, measurementAreas, trajectories240o120o240)
velocities240o120o240 = compute_overall_velocity(times, trajectories240o120o240)

osm_fundamentals240o120o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o120o240)
osm_velocities240o120o240 = compute_overall_velocity(times, osm_trajectories240o120o240)

bhm_fundamentals240o120o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o120o240)
bhm_velocities240o120o240 = compute_overall_velocity(times, bhm_trajectories240o120o240)

gc.collect()

#240-150-240
fundamentals240o150o240_all = compute_fundamentals(times, measurementAreas, trajectories240o150o240)
velocities240o150o240 = compute_overall_velocity(times, trajectories240o150o240)

osm_fundamentals240o150o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o150o240)
osm_velocities240o150o240 = compute_overall_velocity(times, osm_trajectories240o150o240)

bhm_fundamentals240o150o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o150o240)
bhm_velocities240o150o240 = compute_overall_velocity(times, bhm_trajectories240o150o240)

gc.collect()

#240-240-240
fundamentals240o240o240_all = compute_fundamentals(times, measurementAreas, trajectories240o240o240)
velocities240o240o240 = compute_overall_velocity(times, trajectories240o240o240)

osm_fundamentals240o240o240_all = compute_fundamentals(times, measurementAreas, osm_trajectories240o240o240)
osm_velocities240o240o240 = compute_overall_velocity(times, osm_trajectories240o240o240)

bhm_fundamentals240o240o240_all = compute_fundamentals(times, measurementAreas, bhm_trajectories240o240o240)
bhm_velocities240o240o240 = compute_overall_velocity(times, bhm_trajectories240o240o240)

gc.enable()

In [ ]:
def flatten(mylist):
    return functools.reduce(operator.iconcat, mylist, [])

all_fundamentals = [fundamentals240o050o240_all, fundamentals240o060o240_all, fundamentals240o080o240_all, fundamentals240o100o240_all, fundamentals240o120o240_all, fundamentals240o150o240_all, fundamentals240o240o240_all]
osm_all_fundamentals = [osm_fundamentals240o050o240_all, osm_fundamentals240o060o240_all, osm_fundamentals240o080o240_all, osm_fundamentals240o100o240_all, osm_fundamentals240o120o240_all, osm_fundamentals240o150o240_all, osm_fundamentals240o240o240_all]
bhm_all_fundamentals = [bhm_fundamentals240o050o240_all, bhm_fundamentals240o060o240_all, bhm_fundamentals240o080o240_all, bhm_fundamentals240o100o240_all, bhm_fundamentals240o120o240_all, bhm_fundamentals240o150o240_all, bhm_fundamentals240o240o240_all]

fundamentals_all_front = [flatten([e[0][0] for e in all_fundamentals]), flatten([e[0][1] for e in all_fundamentals])]
fundamentals_all_left = [flatten([e[1][0] for e in all_fundamentals]), flatten([e[1][1] for e in all_fundamentals])]
fundamentals_all_right = [flatten([e[2][0] for e in all_fundamentals]), flatten([e[2][1] for e in all_fundamentals])]

osm_fundamentals_all_front = [flatten([e[0][0] for e in osm_all_fundamentals]), flatten([e[0][1] for e in osm_all_fundamentals])]
osm_fundamentals_all_left = [flatten([e[1][0] for e in osm_all_fundamentals]), flatten([e[1][1] for e in osm_all_fundamentals])]
osm_fundamentals_all_right = [flatten([e[2][0] for e in osm_all_fundamentals]), flatten([e[2][1] for e in osm_all_fundamentals])]

bhm_fundamentals_all_front = [flatten([e[0][0] for e in bhm_all_fundamentals]), flatten([e[0][1] for e in bhm_all_fundamentals])]
bhm_fundamentals_all_left = [flatten([e[1][0] for e in bhm_all_fundamentals]), flatten([e[1][1] for e in bhm_all_fundamentals])]
bhm_fundamentals_all_right = [flatten([e[2][0] for e in bhm_all_fundamentals]), flatten([e[2][1] for e in bhm_all_fundamentals])]

# Plot evacuation times

The following plots show the evacuation times for different sceanrios.

In [ ]:
g = sns.catplot(x="scenario", y="evacuationTime", kind="box", hue="model", data=all_evacuation_times, height=8, legend_out=False)
g.set_xticklabels(rotation=20).set_axis_labels("", "Walking times [s]").set(ylim=(0, 40))
handles, labels = g.axes[0][0].get_legend_handles_labels()
g.axes[0][0].legend(handles=handles[0:], labels=labels[0:])
g.savefig('./evacuation_times.pdf', bbox_inches='tight')

# Plot velocities

The following code plots the mean (over all agents / pedestrians) velocity at $t = 0, 0.5, \ldots 70$ and the corresponding standard deviation. 

In [ ]:
fig_velocities = plt.figure(figsize=(15,10))

ax1_velocities = fig_velocities.add_subplot(211)
ax1_velocities.set_xlim(min(times),105)
ax1_velocities.set_ylim(0,2)
#ax1_velocities.set_xlabel("Time [s]")
ax1_velocities.set_ylabel("Mean velocity [m/s]")
ax1_velocities.yaxis.set_label_coords(-0.1, 0.)
ax1_velocities.set_aspect(30)
plt.setp(ax1_velocities.get_xticklabels(), visible=False)
plt.errorbar(times, osm_velocities240o120o240[0], osm_velocities240o120o240[1], label = 'OSM', c = sns.color_palette()[1])
plt.errorbar(times, velocities240o120o240[0], velocities240o120o240[1], label = 'Experiment', c = sns.color_palette()[0])
plt.legend(loc='upper left', bbox_to_anchor=(0.55,1))

ax2_velocities = fig_velocities.add_subplot(212)
ax2_velocities.set_xlim(min(times),105)
ax2_velocities.set_ylim(0,2)
ax2_velocities.set_xlabel("Time [s]")
#ax2_velocities.set_ylabel("Mean velocity [m/s]")
ax2_velocities.set_aspect(30)
plt.errorbar(times, bhm_velocities240o120o240[0], bhm_velocities240o120o240[1], label = 'BHM', c = sns.color_palette()[2])
plt.errorbar(times, velocities240o120o240[0], velocities240o120o240[1], label = 'Experiment', c = sns.color_palette()[0])
plt.legend(loc='upper left', bbox_to_anchor=(0.55,1))
plt.savefig('./mean_velocities.pdf', bbox_inches='tight')

# Plot the velocity of a single agent / pedestrian

In [ ]:
#c_sim_trajecotories
#trajectories240o100o240
traj = get_trajectory(2, trajectories240o050o240)
fig_velocity = plt.figure(figsize=(20,5))
plt.plot(traj['endTime'], traj['velocity'])

# Plot densities inside measurement areas

The following code plots the density inside the measurement area at different times $t$ and the corresponding standard deviation.

In [ ]:
lw = 2
ymin = -0.1
ymax = 2.4
xmin = min(times)
xmax = max(times)
labelx = "Time [s]"

a_ratio = 30
height = 7
width = 3*7

fig_density = plt.figure(figsize=(width,height))
ax1__exp_density = fig_density.add_subplot(131)
ax1__exp_density.set_xlim(xmin, xmax)
ax1__exp_density.set_ylim(ymin,ymax)
ax1__exp_density.set_aspect(a_ratio)
ax1__exp_density.set_xlabel(labelx)
ax1__exp_density.set_ylabel("Density [m$^{-2}$]")
ax1__exp_density.set_title("Experiment")
plt.plot(times, fundamentals240o050o240_all[0][0], linewidth=lw)
plt.plot(times, fundamentals240o100o240_all[0][0], linewidth=lw)
plt.plot(times, fundamentals240o240o240_all[0][0], linewidth=lw)

ax1__osm_density = fig_density.add_subplot(132, sharey=ax1__exp_density)
plt.setp(ax1__osm_density.get_yticklabels(), visible=False)
ax1__osm_density.set_xlim(xmin,xmax)
ax1__osm_density.set_ylim(ymin,ymax)
ax1__osm_density.set_aspect(a_ratio)
ax1__osm_density.set_xlabel(labelx)
ax1__osm_density.set_title("OSM")
plt.plot(times, osm_fundamentals240o050o240_all[0][0], linewidth=lw)
plt.plot(times, osm_fundamentals240o100o240_all[0][0], linewidth=lw)
plt.plot(times, osm_fundamentals240o240o240_all[0][0], linewidth=lw)

ax1__bhm_density = fig_density.add_subplot(133, sharey=ax1__osm_density)
plt.setp(ax1__bhm_density.get_yticklabels(), visible=False)
ax1__bhm_density.set_xlim(xmin,xmax)
ax1__bhm_density.set_ylim(ymin,ymax)
ax1__bhm_density.set_aspect(a_ratio)
ax1__bhm_density.set_xlabel(labelx)
ax1__bhm_density.set_title("BHM")
plt.plot(times, bhm_fundamentals240o050o240_all[0][0], linewidth=lw, label="$b_{in}$ = 50")
plt.plot(times, bhm_fundamentals240o100o240_all[0][0], linewidth=lw, label="$b_{in}$ = 100")
plt.plot(times, bhm_fundamentals240o240o240_all[0][0], linewidth=lw, label="$b_{in}$ = 240")
plt.legend(loc='upper left', bbox_to_anchor=(0.55,1))
plt.savefig('./densities.pdf', bbox_inches='tight')

In [ ]:
lw = 2
ymin = -0.1
ymax = 2.4
xmin = 5
xmax = 100
labelx = "time [s]"

a_ratio = 30
height = 7
width = 2*7


fig = plt.figure(figsize=(width,height))
ax1__density = fig.add_subplot(121)
ax1__density.set_xlim(xmin, xmax)
ax1__density.set_ylim(ymin,ymax)
ax1__density.set_aspect(a_ratio)
ax1__density.set_xlabel(labelx)
ax1__density.set_ylabel("density [m$^{-2}$]")
#ax1__density.set_title("Experiment")
plt.plot(times, fundamentals240o150o240_all[0][0], linewidth=lw, label='Experiment')
plt.plot(times, osm_fundamentals240o150o240_all[0][0], linewidth=lw,label='OSM')
plt.plot(times, bhm_fundamentals240o150o240_all[0][0], linewidth=lw, label='BHM')

ax1_velocity = fig.add_subplot(122)
ax1_velocity.set_xlim(xmin, xmax)
ax1_velocity.set_ylim(ymin,ymax)
ax1_velocity.set_aspect(a_ratio)
ax1_velocity.set_xlabel(labelx)
ax1_velocity.set_ylabel("velocity [m/s]")
#ax1_velocity.set_title("Experiment")
plt.plot(times, fundamentals240o150o240_all[0][1], linewidth=lw, label='Experiment')
plt.plot(times, osm_fundamentals240o150o240_all[0][1], linewidth=lw, label='OSM')
plt.plot(times, bhm_fundamentals240o150o240_all[0][1], linewidth=lw, label='BHM')
plt.legend(loc='upper left', bbox_to_anchor=(0.55,1))
plt.savefig('./densities_velocities.pdf', bbox_inches='tight')

# Plot velocities inside measurement areas

The following code plots the velocity inside the measurement area at different times $t$ and the corresponding standard deviation.

In [ ]:
lw = 2
ymin = -0.1
ymax = 2.0
xmin = min(times)
xmax = max(times)
labelx = "Time [s]"

a_ratio = 50
height = 7
width = 3*7

fig_density = plt.figure(figsize=(width,height))
ax1__exp_density = fig_density.add_subplot(131)
ax1__exp_density.set_xlim(xmin, xmax)
ax1__exp_density.set_ylim(ymin,ymax)
ax1__exp_density.set_aspect(a_ratio)
ax1__exp_density.set_xlabel(labelx)
ax1__exp_density.set_ylabel("Velocity [m/s]")
ax1__exp_density.set_title("Experiment")
plt.plot(times, fundamentals240o050o240_all[0][1], linewidth=lw)
plt.plot(times, fundamentals240o100o240_all[0][1], linewidth=lw)
plt.plot(times, fundamentals240o240o240_all[0][1], linewidth=lw)

ax1__osm_density = fig_density.add_subplot(132, sharey=ax1__exp_density)
plt.setp(ax1__osm_density.get_yticklabels(), visible=False)
ax1__osm_density.set_xlim(xmin,xmax)
ax1__osm_density.set_ylim(ymin,ymax)
ax1__osm_density.set_aspect(a_ratio)
ax1__osm_density.set_xlabel(labelx)
ax1__osm_density.set_title("OSM")
plt.plot(times, osm_fundamentals240o050o240_all[0][1], linewidth=lw)
plt.plot(times, osm_fundamentals240o100o240_all[0][1], linewidth=lw)
plt.plot(times, osm_fundamentals240o240o240_all[0][1], linewidth=lw)

ax1__bhm_density = fig_density.add_subplot(133, sharey=ax1__osm_density)
plt.setp(ax1__bhm_density.get_yticklabels(), visible=False)
ax1__bhm_density.set_xlim(xmin,xmax)
ax1__bhm_density.set_ylim(ymin,ymax)
ax1__bhm_density.set_aspect(a_ratio)
ax1__bhm_density.set_xlabel(labelx)
ax1__bhm_density.set_title("BHM")
plt.plot(times, bhm_fundamentals240o050o240_all[0][1], linewidth=lw, label="$b_{in}$ = 50")
plt.plot(times, bhm_fundamentals240o100o240_all[0][1], linewidth=lw, label="$b_{in}$ = 100")
plt.plot(times, bhm_fundamentals240o240o240_all[0][1], linewidth=lw, label="$b_{in}$ = 240")
plt.legend(loc='upper left', bbox_to_anchor=(0.55,1))
plt.savefig('./velocities.pdf', bbox_inches='tight')

# Plot of fundamental diagrams using method c

## Front

In [ ]:
ymin = -0.1
ymax = 2.1
xmin = -0.2
xmax = 2.5
ms = 7
labelx = "Density [m$^{-2}$]"
a_ratio = 2
height = 7
width = 3*7

fig_fundamental = plt.figure(figsize=(10,5))
ax_exp_trajectories = fig_fundamental.add_subplot(131)
ax_exp_trajectories.set_ylim(ymin,ymax)
ax_exp_trajectories.set_xlim(xmin,xmax)
ax_exp_trajectories.set_aspect(a_ratio)
ax_exp_trajectories.set_xlabel(labelx)
ax_exp_trajectories.set_ylabel("Velocity [m/s]")
ax_exp_trajectories.set_title("Experiment")

plt.plot(fundamentals240o050o240_all[0][0], fundamentals240o050o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o060o240_all[0][0], fundamentals240o060o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o080o240_all[0][0], fundamentals240o080o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o100o240_all[0][0], fundamentals240o100o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o120o240_all[0][0], fundamentals240o120o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o150o240_all[0][0], fundamentals240o150o240_all[0][1] ,'.',markersize=ms)
plt.plot(fundamentals240o240o240_all[0][0], fundamentals240o240o240_all[0][1] ,'.',markersize=ms)

ax_osm_trajectories = fig_fundamental.add_subplot(132)
plt.setp(ax_osm_trajectories.get_yticklabels(), visible=False)
ax_osm_trajectories.set_ylim(ymin,ymax)
ax_osm_trajectories.set_xlim(xmin,xmax)
ax_osm_trajectories.set_aspect(a_ratio)
ax_osm_trajectories.set_xlabel(labelx)
ax_osm_trajectories.set_title("OSM")
#ax_osm_trajectories.set_ylabel("velocity")

plt.plot(osm_fundamentals240o050o240_all[0][0], osm_fundamentals240o050o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o060o240_all[0][0], osm_fundamentals240o060o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o080o240_all[0][0], osm_fundamentals240o080o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o100o240_all[0][0], osm_fundamentals240o100o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o120o240_all[0][0], osm_fundamentals240o120o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o150o240_all[0][0], osm_fundamentals240o150o240_all[0][1] ,'.',markersize=ms)
plt.plot(osm_fundamentals240o240o240_all[0][0], osm_fundamentals240o240o240_all[0][1] ,'.',markersize=ms)

ax_bhm_trajectories = fig_fundamental.add_subplot(133)
plt.setp(ax_bhm_trajectories.get_yticklabels(), visible=False)
ax_bhm_trajectories.set_ylim(ymin,ymax)
ax_bhm_trajectories.set_xlim(xmin,xmax)
ax_bhm_trajectories.set_aspect(a_ratio)
ax_bhm_trajectories.set_xlabel(labelx)
ax_bhm_trajectories.set_title('BHM')

plt.plot(bhm_fundamentals240o050o240_all[0][0], bhm_fundamentals240o050o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 50")
plt.plot(bhm_fundamentals240o060o240_all[0][0], bhm_fundamentals240o060o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 60")
plt.plot(bhm_fundamentals240o080o240_all[0][0], bhm_fundamentals240o080o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 80")
plt.plot(bhm_fundamentals240o100o240_all[0][0], bhm_fundamentals240o100o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 100")
plt.plot(bhm_fundamentals240o120o240_all[0][0], bhm_fundamentals240o120o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 120")
plt.plot(bhm_fundamentals240o150o240_all[0][0], bhm_fundamentals240o150o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 150")
plt.plot(bhm_fundamentals240o240o240_all[0][0], bhm_fundamentals240o240o240_all[0][1] ,'.',markersize=ms, label="$b_{in}$ = 240")

plt.legend(loc=2, bbox_to_anchor=(1.05, 1), borderaxespad=0.)

plt.savefig('./fundamental_front.pdf', bbox_inches='tight')
plt.show()

## Combined

In [ ]:
ymin = -0.1
ymax = 2.1
xmin = -0.2
xmax = 4
ms = 5
labelx = "Density [#peds / m$^{2}$]"
a_ratio = 2
height = 7
width = 3*7
palette=sns.color_palette()

fig_fundamental = plt.figure(figsize=(width,height))
ax_exp_trajectories = fig_fundamental.add_subplot(131)
ax_exp_trajectories.set_ylim(ymin,ymax)
ax_exp_trajectories.set_xlim(xmin,xmax)
ax_exp_trajectories.set_aspect(a_ratio)
#ax_exp_trajectories.set_xlabel(labelx)
#ax_exp_trajectories.xaxis.set_label_coords(1.5, -0.1)
ax_exp_trajectories.set_ylabel("Velocity [m/s]")
ax_exp_trajectories.set_title("Experiment")

plt.plot(fundamentals_all_right[0], fundamentals_all_right[1], '^', label="T-right", markersize=ms, c=palette[4])
plt.plot(fundamentals_all_left[0], fundamentals_all_left[1], 'd', label="T-left", markersize=ms, c=palette[8])
plt.plot(fundamentals_all_front[0], fundamentals_all_front[1] ,'o', label="T-top", markersize=ms, c=palette[9])


ax_osm_trajectories = fig_fundamental.add_subplot(132)
ax_osm_trajectories.set_ylim(ymin,ymax)
ax_osm_trajectories.set_xlim(xmin,xmax)
ax_osm_trajectories.set_aspect(a_ratio)
ax_osm_trajectories.set_xlabel(labelx)
#ax_osm_trajectories.set_ylabel("velocity")
ax_osm_trajectories.set_title("OSM")

plt.setp(ax_osm_trajectories.get_yticklabels(), visible=False)
plt.plot(osm_fundamentals_all_right[0], osm_fundamentals_all_right[1], '^', label="R-right", markersize=ms, c=palette[4])
plt.plot(osm_fundamentals_all_left[0], osm_fundamentals_all_left[1], 'd', label="T-left",markersize=ms, c=palette[8])
plt.plot(osm_fundamentals_all_front[0], osm_fundamentals_all_front[1] ,'o', label="T-top",markersize=ms, c=palette[9])

ax_bhm_trajectories = fig_fundamental.add_subplot(133)
ax_bhm_trajectories.set_ylim(ymin,ymax)
ax_bhm_trajectories.set_xlim(xmin,xmax)
ax_bhm_trajectories.set_aspect(a_ratio)
#ax_bhm_trajectories.set_xlabel(labelx)
#ax_bhm_trajectories.set_ylabel("velocity")
ax_bhm_trajectories.set_title("BHM")

plt.setp(ax_bhm_trajectories.get_yticklabels(), visible=False)
plt.plot(bhm_fundamentals_all_right[0], bhm_fundamentals_all_right[1], '^', label="right", markersize=ms, c=palette[4])
plt.plot(bhm_fundamentals_all_left[0], bhm_fundamentals_all_left[1], 'd', label="left", markersize=ms, c=palette[8])
plt.plot(bhm_fundamentals_all_front[0], bhm_fundamentals_all_front[1] ,'o', label="top", markersize=ms, c=palette[9])

plt.legend(loc='upper left', bbox_to_anchor=(0.6,1))
plt.savefig('./fundamental_all.pdf', bbox_inches='tight')
plt.show()

In [ ]:
palette[8]